In [228]:
model_file = "./model/LSTMfuture_v4.weight"
#testing_user = "./data/testing_users.csv"
testing_user = "./clean_data/test_data/test_before_021718_7000row_fold0.csv"
submission_file = "./submission/deep_learning_lstm_v1.csv"

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence


torch.cuda.set_device(1)
print(torch.cuda.current_device())

1


In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

modelwv = Doc2Vec.load("./model/doc2vec_before_train.wv")

## Predicting projects for user using deep learning

In [3]:
import pandas as pd
import datetime
import torch

In [4]:
# Get all necessary data frame here
submission_users = pd.read_csv(testing_user)
userLog = pd.read_csv('./clean_data/userLog_beforeTest.csv')
userLog.head()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,datetime
0,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018-01-01 00:00:00
1,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018-01-01 00:00:00
2,cb5b4b68-cc01-6db6-f54b-4a0f881301c5,5067,Mobile,iPhone,HomeWebsite,2018-01-01 00:00:00
3,5f74cef2-0d1e-b619-3564-0955a14e0985,6654,Mobile,iPhone,Google,2018-01-01 00:00:00
4,dba8f279-844e-eef6-73ac-22bd7d1353cc,6474,Mobile,iPad,Google,2018-01-01 00:00:00


### Map project id to project Index [0 to index]

In [5]:
modelwv.wv.index2word.index('6709')

userLog = pd.read_csv("./clean_data/userlog_feature.csv")#.head(100000)
userLog['datetime'] = pd.to_datetime(userLog['datetime'])
isTrain = userLog['datetime']<datetime.datetime(2018,2,18)
test = userLog[~isTrain]
userLog = userLog[isTrain]
userLog1 = userLog.sort_values("datetime")


In [6]:
userLogSeq = userLog1.groupby("userCode").agg({"project_id":lambda x: list(x)})
userLogSeqFiltered = userLogSeq[userLogSeq['project_id'].apply(lambda x: len(x))>1]

In [208]:
# Clean user df
submission_users_merged = submission_users.merge(userLogSeqFiltered['project_id'].reset_index(), on='userCode')
#submission_users.columns = 'userCode',' ans_project_id',' project_id'
submission_users_merged['ans_project_id'] = submission_users_merged['ans_project_id'].apply(eval)
submission_users_merged['project_id'] = submission_users_merged['project_id'].apply(lambda x: [modelwv.wv.index2word.index(str(i)) for i in x])
submission_users_merged['ans_project_id'] = submission_users_merged['ans_project_id'].apply(lambda x: [modelwv.wv.index2word.index(str(i)) for i in x if str(i) in modelwv.wv.index2word])

In [209]:
submission_users_merged = submission_users_merged.set_index("userCode")

## Begin padding before input to LSTM

In [210]:
submission_users_merged['seq_len'] = submission_users_merged['project_id'].apply(len)
filtered_submission_users = submission_users_merged[submission_users_merged['seq_len']<500]
userSeq = filtered_submission_users.sort_values('seq_len',ascending=False)
maxSeq = max(userSeq['seq_len'])

In [211]:
userSeq['padded_features'] = userSeq.apply(lambda x: x['project_id']+[0]*(maxSeq - x['seq_len']),axis=1)

In [212]:
userSeq['new_len'] = userSeq['padded_features'].apply(len)
userSeq['new_len'].value_counts()

267    4699
Name: new_len, dtype: int64

In [213]:
userSeq['padded_features'].head()

userCode
4c42709a-19f6-056a-b504-1e402dbca126    [4534, 690, 690, 2933, 690, 773, 3182, 530, 34...
1aecd8e5-f02b-ba5c-e865-49da43ad8f22    [4633, 4633, 4985, 5105, 5106, 3040, 5107, 510...
d8ebf128-caa2-b3d7-0bea-8fbb9fb9eb87    [44, 157, 44, 201, 13, 13, 129, 564, 201, 798,...
334963e3-56e0-8fed-2a80-460e6bec516e    [259, 259, 891, 104, 1209, 104, 2208, 94, 1487...
a5021c5b-f424-e5f9-4675-cff0d0e302fb    [3707, 5089, 4645, 5270, 5053, 5271, 4995, 387...
Name: padded_features, dtype: object

In [214]:
features = Variable(torch.Tensor(userSeq['padded_features']).long())#.cuda())
seq_len = torch.Tensor(userSeq['seq_len'].values).long()#.cuda()

print(features.shape)

torch.Size([4699, 267])


# Now we are ready

In [215]:
def from_pretrained(embeddings, freeze=True):
    assert embeddings.dim() == 2, \
         'Embeddings parameter is expected to be 2-dimensional'
    rows, cols = embeddings.shape
    embedding = torch.nn.Embedding(num_embeddings=rows, embedding_dim=cols)
    embedding.weight = torch.nn.Parameter(embeddings)
    embedding.weight.requires_grad = not freeze
    return embedding


weights = torch.FloatTensor(modelwv.wv.syn0)
weights.shape[0]

from_pretrained(weights, False)

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  # Remove the CWD from sys.path while we load stuff.


Embedding(5317, 300)

In [216]:
hidden_size = 30

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        #self.projEmbedding = nn.Embedding(*weights.shape).from_pretrained(weight) # Spreading tuple into embedding dim
        self.projEmbedding = from_pretrained(weights, False)
        
        self.lstm = nn.LSTM(weights.shape[1],hidden_size, bidirectional = False)
        
        self.hidden2out = nn.Linear(hidden_size, weights.shape[0])#weights.shape[0])
        #self.init_hidden()
    
    def init_hidden(self, batch_size):
        #TODO: change to clicking context
        self.hidden = (Variable(torch.zeros(1, batch_size, hidden_size)),Variable(torch.zeros(1, batch_size, hidden_size)))
        
    def forward(self, inputs, lengths):

        batch_size = inputs.size()[0]
        
        embeds = self.projEmbedding(inputs)
        
        transposed_embeds = embeds.transpose(0,1)
        
        self.init_hidden(batch_size)
        
        packed_input = pack_padded_sequence(transposed_embeds, lengths.numpy())
        packed_outputs, (ht, ct) = self.lstm(packed_input, self.hidden)
        
        out, _ = pad_packed_sequence(packed_outputs)
        
        out = self.hidden2out(out)
      
        
        return out.transpose(0,1)#out.transpose(0,1)#


In [229]:
model = LSTM()
model.load_state_dict(torch.load(model_file))

In [219]:
outputs = model(features[:5], seq_len[:5])
outputs.shape

torch.Size([5, 267, 5317])

In [221]:
# TODO extract most probably correct projects from deep

def score_AP(correct, recommended, n=7):
    recommended = list(recommended)[:7]
    assert(len(set(recommended))==len(recommended)) # Recommended should be unique    
    precisions = []
    correct_score = 1
    for idx, recommend_item in enumerate(recommended):
        if str(recommend_item) not in correct: precisions.append(0)
        else:                         
            precisions.append(correct_score/(idx+1))
            correct_score+=1
    return sum(precisions)/min(len(correct),n)
#score_AP(["5858", "5678"],"5678 5858 8058 8107 6527 9132 6446",7)

In [230]:
#Get latest prediction
#def 
n = 4700
outputs = model(features[:n], seq_len[:n])
preds = np.array([o[seq-1].data.numpy() for o, seq in zip(outputs[:n],seq_len[:n])])
idxes = [(-preds[1]).argsort()[:30] for pred in preds]
filtered_submission_users['preds'] = idxes

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [231]:
eval_df = filtered_submission_users[filtered_submission_users['ans_project_id'].apply(len)>0]
eval_df['MAP'] = eval_df.apply(lambda x: score_AP(x['ans_project_id'],x['preds']),axis=1)

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [232]:
eval_df.head().sum()

ans_project_id        [79, 540, 320, 1500, 302, 3528, 97, 17, 1119]
project_id        [502, 48, 473, 473, 184, 1296, 1052, 931, 931,...
seq_len                                                          31
preds             [75, 110, 85, 25, 0, 95, 90, 35, 20, 55, 5, 24...
MAP                                                               0
dtype: object

In [240]:
userSeq.head()

,ans_project_id,project_id,seq_len,padded_features,new_len
userCode,,,,,
4c42709a-19f6-056a-b504-1e402dbca126,"[2953, 4346, 4161]","[4534, 690, 690, 2933, 690, 773, 3182, 530, 34...",267,"[4534, 690, 690, 2933, 690, 773, 3182, 530, 34...",267
1aecd8e5-f02b-ba5c-e865-49da43ad8f22,"[702, 5038, 5312, 3789, 2013, 2524, 4494, 5049...","[4633, 4633, 4985, 5105, 5106, 3040, 5107, 510...",244,"[4633, 4633, 4985, 5105, 5106, 3040, 5107, 510...",267
d8ebf128-caa2-b3d7-0bea-8fbb9fb9eb87,"[2945, 1246, 426]","[44, 157, 44, 201, 13, 13, 129, 564, 201, 798,...",194,"[44, 157, 44, 201, 13, 13, 129, 564, 201, 798,...",267
334963e3-56e0-8fed-2a80-460e6bec516e,"[159, 187, 664]","[259, 259, 891, 104, 1209, 104, 2208, 94, 1487...",188,"[259, 259, 891, 104, 1209, 104, 2208, 94, 1487...",267
a5021c5b-f424-e5f9-4675-cff0d0e302fb,"[1772, 1198, 1951, 5127, 620, 642, 4404, 4331,...","[3707, 5089, 4645, 5270, 5053, 5271, 4995, 387...",186,"[3707, 5089, 4645, 5270, 5053, 5271, 4995, 387...",267


In [241]:
userSeq.to_csv("./clean_data/deeplstmfeatures.csv")

In [237]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))